In [56]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import threading
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from queue import Queue


class Croma:
    def __init__(self, search, limit, page_visit):  # , count_product
        self.search = search
        self.limit = limit
        self.page_visit = page_visit
        #self.count_product = count_product
        self.result_queue = Queue()
        self.s = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service=self.s)
        self.driver.get('https://www.croma.com/')
        self.flag = False

        while self.flag != True:
            try:
                # self.un = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME,'MuiAutocomplete-input.MuiAutocomplete-inputFocused.search-field')))
                self.un = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="searchV2"]')))

                self.un.send_keys(self.search)
                self.flag = True
            except (ElementNotInteractableException, TimeoutException):
                self.driver.refresh()
                time.sleep(3)
                pass
        #                 self.un = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME,'MuiAutocomplete-input.MuiAutocomplete-inputFocused.search-field')))
        #                 self.un.send_keys(self.search)
        #                 self.flag = True

        time.sleep(2)
        self.un.send_keys(Keys.ENTER)

    #         try:
    #             self.un = WebDriverWait(self.driver, 12).until(EC.presence_of_element_located((By.CLASS_NAME,'MuiAutocomplete-input.MuiAutocomplete-inputFocused.search-field')))
    #             #time.sleep(2)
    #             self.un.send_keys(search)
    #             time.sleep(2)
    #             self.un.send_keys(Keys.ENTER)
    #         except ElementNotInteractableException:
    #             self.driver.refresh()
    #             self.un = WebDriverWait(self.driver, 12).until(EC.presence_of_element_located((By.CLASS_NAME,'MuiAutocomplete-input.MuiAutocomplete-inputFocused.search-field')))
    #             #time.sleep(2)
    #             self.un.send_keys(search)
    #             time.sleep(2)
    #             self.un.send_keys(Keys.ENTER)

    def link_collection(self):

        self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
        time.sleep(2)
        
        link_texts = []
        
        te = self.search.split(' ')
        #y = []
        #print(x)
        for j in te:
            k = 0
            for i in range(len(j)):
                modified_string = f"{j[:i]}{j[i].upper()}{j[i+1:]}"
                link_texts.append(modified_string)
                k+=1
                if k==2:
                    break
                    
                    
        link = []
        for i in link_texts:
            try:
                review_link = WebDriverWait(self.driver, 5).until(EC.presence_of_all_elements_located((By.PARTIAL_LINK_TEXT, i)))
                if len(review_link) != 0:
                    for i in review_link:
                        link.append(i)
            except TimeoutException:
                pass
        #print(link)
        
#         review_link = WebDriverWait(self.driver, 10).until(
#             EC.presence_of_all_elements_located((By.PARTIAL_LINK_TEXT, self.search)))
        # name = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'_4rR01T')))

        d = {}
        name = []
        for i in link:
            name.append(i.text)
            #print(i.text)
        #print(name)
        for i, j in zip(link, name):
            #x = self.search.lower()
            #if x in j.lower() and '(Renewed)' not in j:
            d[i] = j

        time.sleep(1)
        #print(d)
        return d

    def scrapings(self, d, limit): # , count_product
        j = 1
        prod_review = {}
        no_of_product  = 0
        link = {}
        price = {}
        ratings = {}
        avg_rating = {}

        for i in d:
            # if no_of_product == self.count_product:
            #     break
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            self.driver.switch_to.window(self.driver.window_handles[0])
            # i.click()
            webdriver.ActionChains(self.driver).move_to_element(i).click(i).perform()
            while len(self.driver.window_handles) != 2:
                time.sleep(3)
                webdriver.ActionChains(self.driver).move_to_element(i).click(i).perform()

            # driver.execute_script("arguments[0].click();", i)
            time.sleep(2)
            # driver.switch_to.window(driver.window_handles[j])
            c = self.driver.window_handles[1]
            self.driver.switch_to.window(c)

            y = d[i]
            # scraping link
            l = self.driver.current_url
            link[y] = l

            # scraping price
            e = self.driver.page_source
            x = BeautifulSoup(e, 'html.parser')
            p = []
            h = x.find_all('span', {'class': 'amount'})

            if len(h) <= 0:
                price[y] = p
            else:
                p.append(h[0].text)
                price[y] = p

            # time.sleep(2)
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            time.sleep(7)
            # try:
            #    all_comments = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME,'btn-wrap.view-all-review-btn')))
            # all_comments[0].click()
            #   webdriver.ActionChains(driver).move_to_element(all_comments).click(all_comments).perform
            # except TimeoutException:
            #    pass

            # scrapping ratings
            e = self.driver.page_source
            x = BeautifulSoup(e, 'html.parser')

            r = []
            rating = {'5 Stars': 0, '4 Stars': 0, '3 Stars': 0, '2 Stars': 0, '1 Stars': 0}
            l = h = x.find_all('div', {'class': 'barAndStar'})
            # h = x.find_all('p', {'class' : 'right-text'})
            # rating['4 Stars'] = l[0].text
            if len(l) <= 0:
                ratings[y] = rating
            else:
                count = 5
                for i in l:
                    rating['{} Stars'.format(count)] = i.text[6:]
                    count -= 1

                ratings[y] = rating

            # scraping  avg ratings
            e = self.driver.page_source
            x = BeautifulSoup(e, 'html.parser')
            t = []
            h = x.find_all('div', {'class': 'totalRating'})

            if len(h) <= 0:
                avg_rating[y] = t
            else:
                t.append(h[0].text)
                avg_rating[y] = t

            # time.sleep(3)

            # scraping review
            e = self.driver.page_source
            x = BeautifulSoup(e, 'html.parser')

            h = x.find_all('div', {'class': 'desc'})
            reviews = set()
            # y = d[i]
            for k in h:
                reviews.add(k.text)
                print(k.text)
                if len(reviews) == self.limit:
                    break
            # driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            # curr_height = driver.execute_script('return document.body.scrollHeight;')
            flag = 0
            reviews1 = reviews
            while len(reviews) != self.limit:
                try:
                    # driver.execute_script('window.scrollBy(0, 2000);')
                    # time.sleep(2)
                    flag += 1
                    more = WebDriverWait(self.driver, 10).until(
                        EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, 'View All Reviews')))
                    webdriver.ActionChains(self.driver).move_to_element(more).click(more).perform()
                    e = self.driver.page_source
                    x = BeautifulSoup(e, 'html.parser')
                    t = x.find_all('div', {'class': 'desc'})
                    # print(curr_height)
                    # curr_height += 100
                    for l in t:
                        reviews.add(l.text.strip())
                        if len(reviews) == self.limit:
                            break

                    self.driver.execute_script('window.scrollBy(5350, 100);')
                    time.sleep(3)
                    print(flag)
                    if flag == 11:
                        if len(reviews1) == len(reviews):
                            break

                    # driver.execute_script('window.scrollBy(0, 2000);')
                except TimeoutException:
                    flag += 3
                    if flag >= 11:
                        break
                    else:
                        pass

            prod_review[y] = reviews
            self.driver.close()
            self.driver.switch_to.window(self.driver.window_handles[0])
            no_of_product +=1
            print(j)
            j += 1

        return prod_review, ratings, link, avg_rating, price

    def run(self, page_visit):
        links = []
        reviews = []
        all_ratings = []
        prices = []
        avg_r = []

        main_d = {}
        now = time.time()
        for i in range(self.page_visit):
            d = self.link_collection()
            print(len(d))
            new_d = {}
            for i, j in d.items():
                if i in main_d:
                    continue
                else:
                    new_d[i] = j
            for i, j in d.items():
                main_d[i] = j
            x, y, z, a, b = self.scrapings(new_d, self.limit) #  , self.count_product
            reviews.append(x)
            all_ratings.append(y)
            links.append(z)
            avg_r.append(a)
            prices.append(b)

            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            time.sleep(3)
            next_page = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'btn.btn-secondary.btn-viewmore')))
            webdriver.ActionChains(self.driver).move_to_element(next_page).click(next_page).perform()

        self.result_queue.put((reviews, prices, avg_r, all_ratings, links))

        # return reviews, all_ratings, links, avg_r, prices
# x = Croma('iPhone 14', 10)




In [57]:
x = Croma('earbuds under 2000', 10, 1)
x.run(1) # , data4

['pTron Bassbuds Air 140318669 TWS Earbuds with Passive Noise Cancellation (IPX4 Water Resistant, Touch Control, Black)', 'boAt Airdopes 148 TWS Earbuds (IPX4 Sweat & Water Resistant, 42 Hours Playtime, Cyan Cider)', 'realme Techlife RMA2109 Earbuds with AI Environment Noise Cancellation (IPX5 Water Resistant, Google Fast Pair, Black)', 'pTron Bassbuds Air 140318670 TWS Earbuds with Passive Noise Cancellation (IPX4 Water Resistant, Touch Control, Blue)', 'boAt Airdopes 131 Pro TWS Earbuds with Environmental Noise Cancellation Technology (IPX5 Water Resistant, ASAP Charge, Black)', 'realme Air 3 Neo RMA2113 Earbuds with AI Environmental Noise Cancellation (IPX5 Water Resistant, Bluetooth Connectivity, Starry Blue)', 'boAt Airdopes 163 TWS Earbuds (IPX5 Water Resistant, IWP Technology, Active Black)', 'boAt Airdopes 138 TWS Earbuds (IPX4 Water & Dust Resistant, 12 Hours Playback, Black)', 'Noise Buds Smart AUD-HDPHN-BUDSSMAR TWS Earbuds (IPX5 Water Resistant, 18 Hours Playback, Jet Black

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.198)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x004DA813+48355]
	(No symbol) [0x0046C4B1]
	(No symbol) [0x00375358]
	(No symbol) [0x0035D293]
	(No symbol) [0x003BE37B]
	(No symbol) [0x003CC473]
	(No symbol) [0x003BA536]
	(No symbol) [0x003982DC]
	(No symbol) [0x003993DD]
	GetHandleVerifier [0x0073AABD+2539405]
	GetHandleVerifier [0x0077A78F+2800735]
	GetHandleVerifier [0x0077456C+2775612]
	GetHandleVerifier [0x005651E0+616112]
	(No symbol) [0x00475F8C]
	(No symbol) [0x00472328]
	(No symbol) [0x0047240B]
	(No symbol) [0x00464FF7]
	BaseThreadInitThunk [0x76DDFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77D57B5E+286]
	RtlGetAppContainerNamedObjectPath [0x77D57B2E+238]


In [42]:
s = Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service = s)
driver.get('https://www.croma.com/')

In [ ]:
time.sleep(2)
review_link = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.PARTIAL_LINK_TEXT, search)))
#name = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'_4rR01T')))


d = {}
name = []
for i in review_link:
    name.append(i.text)

for i, j in  zip(review_link, name):
    x = search.lower()
    if x in j.lower() and '(Renewed)' not in j:
        d[i] = j

time.sleep(1)
return d

In [38]:
x = search.split(' ')
y = []
print(x)
for j in x:
    k = 0
    for i in range(len(j)):
        modified_string = f"{j[:i]}{j[i].upper()}{j[i+1:]}"
        y.append(modified_string)
        k+=1
        if k==2:
            break
            
y        

['earbuds', 'under', '2000']


['Earbuds', 'eArbuds', 'Under', 'uNder', '2000', '2000']

In [39]:
link = []
for i in y:
    try:
        review_link = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.PARTIAL_LINK_TEXT, i)))
        if len(review_link) != 0:
            for i in review_link:
                link.append(i)
    except TimeoutException:
        pass
link

[<selenium.webdriver.remote.webelement.WebElement (session="d8aa11153a0e24498811e737126e4f16", element="C5121CFF054CE6738E1025CFD16F8968_element_301")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d8aa11153a0e24498811e737126e4f16", element="C5121CFF054CE6738E1025CFD16F8968_element_320")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d8aa11153a0e24498811e737126e4f16", element="C5121CFF054CE6738E1025CFD16F8968_element_167")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d8aa11153a0e24498811e737126e4f16", element="C5121CFF054CE6738E1025CFD16F8968_element_168")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d8aa11153a0e24498811e737126e4f16", element="C5121CFF054CE6738E1025CFD16F8968_element_321")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d8aa11153a0e24498811e737126e4f16", element="C5121CFF054CE6738E1025CFD16F8968_element_330")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d8aa11153a0e24498811e7

In [6]:
search = input()

earbuds under 2000


In [16]:
review_link = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.PARTIAL_LINK_TEXT, 'boAt')))


[<selenium.webdriver.remote.webelement.WebElement (session="d8aa11153a0e24498811e737126e4f16", element="C5121CFF054CE6738E1025CFD16F8968_element_320")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d8aa11153a0e24498811e737126e4f16", element="C5121CFF054CE6738E1025CFD16F8968_element_321")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d8aa11153a0e24498811e737126e4f16", element="C5121CFF054CE6738E1025CFD16F8968_element_322")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d8aa11153a0e24498811e737126e4f16", element="C5121CFF054CE6738E1025CFD16F8968_element_323")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d8aa11153a0e24498811e737126e4f16", element="C5121CFF054CE6738E1025CFD16F8968_element_324")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d8aa11153a0e24498811e737126e4f16", element="C5121CFF054CE6738E1025CFD16F8968_element_325")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d8aa11153a0e24498811e7